In [2]:
!pip install pandas
!pip install pypdf
!pip install pypdf2
!pip install cryptography
!pip install pikepdf

In [2]:
import os
import shutil
import pikepdf
import re
import pandas as pd
from PyPDF2 import PdfReader

# Caminhos das pastas
source_folder = "./maps/encrypted/"
processed_folder = os.path.join(source_folder, "processed")
decrypted_folder = "./maps/decrypted/"
decrypted_processed_folder = os.path.join(decrypted_folder, "processed")

# Função para verificar e criar a hierarquia de pastas
def ensure_folders_exist():
    folders = [source_folder, processed_folder, decrypted_folder, decrypted_processed_folder]
    for folder in folders:
        if not os.path.exists(folder):
            os.makedirs(folder)
            print(f"Pasta criada: {folder}")
        else:
            print(f"Pasta já existe: {folder}")

# Cria a hierarquia de pastas
ensure_folders_exist()


# Busca o primeiro arquivo PDF na pasta maps
pdf_files = [f for f in os.listdir(source_folder) if f.endswith('.pdf')]

if not pdf_files:
    print("Nenhum arquivo PDF encontrado na pasta maps.")
else:
    encrypted_pdf_path = os.path.join(source_folder, pdf_files[0])
    decrypted_pdf_path = os.path.join(decrypted_folder, f"decrypted_{pdf_files[0]}")

    try:
        # Tenta descriptografar o PDF
        with pikepdf.open(encrypted_pdf_path) as pdf:
            pdf.save(decrypted_pdf_path)
        print(f"PDF desbloqueado com sucesso! Salvo em: {decrypted_pdf_path}")

        # Move o PDF criptografado para a pasta processed
        shutil.move(encrypted_pdf_path, os.path.join(processed_folder, pdf_files[0]))
        print(f"PDF original criptografado movido para: {processed_folder}")

    except pikepdf.PasswordError:
        print(f"O PDF '{pdf_files[0]}' está protegido por senha e não pode ser desbloqueado sem ela.")
    except FileNotFoundError as e:
        print(f"Erro: Arquivo não encontrado. Verifique o caminho. {e}")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")




# Paths das pastas
input_folder = "./maps/decrypted"
processed_folder = "maps/decrypted/processed"

# Verificar se a pasta de processados existe, caso contrário, criá-la
os.makedirs(processed_folder, exist_ok=True)

# Obter a lista de PDFs na pasta de entrada
pdf_files = [file for file in os.listdir(input_folder) if file.lower().endswith(".pdf")]

if pdf_files:
    # Processar cada PDF na pasta
    for file_name in pdf_files:
        input_pdf_path = os.path.join(input_folder, file_name)
        processed_pdf_path = os.path.join(processed_folder, file_name)
        
        try:
            # Inicializar o texto completo para o PDF atual
            full_text = ""
            
            # Ler o PDF
            reader = PdfReader(input_pdf_path)
            
            # Iterar sobre as páginas e adicionar os delimitadores
            for idx, page in enumerate(reader.pages):
                full_text += f"------------------ PAGINA {idx} ----------------\n"
                full_text += page.extract_text()
            
            # Dividir o texto baseado nos delimitadores
            paginas = re.split(r"------------------ PAGINA \d+ ----------------", full_text)
            paginas = [pagina.strip() for pagina in paginas if pagina.strip()]
            
            # Exibir a quantidade de páginas úteis detectadas
            print(f"\nTotal de páginas detectadas em '{file_name}': {len(paginas)}\n")
            
            # Criar dicionário para armazenar o conteúdo das páginas
            texto_paginas = {}
            for idx, pagina in enumerate(paginas, start=1):
                texto_paginas[f"texto_pagina{idx}"] = pagina
            
        except Exception as e:
            print(f"Ocorreu um erro ao processar '{file_name}': {e}")
        finally:
            # Mover o PDF processado para a pasta de processados
            shutil.move(input_pdf_path, processed_pdf_path)
            print(f"PDF '{file_name}' movido para a pasta de processados: {processed_pdf_path}")
else:
    print("Nenhum PDF encontrado na pasta de entrada.")



# Regexes para capturar os elementos de interesse
regexes = {
    'nome': re.compile(r'Nome:\s+(.+)', re.MULTILINE),
    'nif': re.compile(r'Nº de Identificação:\s+(\d+)'),
    'mes_mapa': re.compile(r'Responsabilidades de crédito referentes a\s+(.+)'),
    'instituicao': re.compile(r'Informação comunicada pela instituição:\s+(.+)'),
    'total_em_divida': re.compile(r"Montantes\s+Total em dívida\s+do qual, em incumprimento\s+([\d\s,]+) €"),
    'litigio': re.compile(r'Em litígio judicial\s+(Sim|Não)'),
    'abatido_ativo': re.compile(r'Abatido ao ativo\s+([\d\s,.]+) €'),
    'garantias': re.compile(r"Tipo\s+Valor\s+Número\s+\d+\s+([\d\s,.]+) €"),
    'num_devedores': re.compile(r"Nº devedores no contrato\s+(\d+)"),
    'prod_financeiro': re.compile(r"Produto financeiro\s+(.+?)\s+Tipo de responsabilidade"),
    'dat_inicio': re.compile(r"Início\s+(\d{4}-\d{2}-\d{2})"),
    'dat_fim': re.compile(r"Fim\s+(\d{4}-\d{2}-\d{2})\s+Em litígio judicial"),
    'entrada_incumpr': re.compile(r"Entrada incumpr\.\s+(\d{4}-\d{2}-\d{2})\s+Tipo")
}

# Função para aplicar regex em texto
def get_feature(text, regex_string):
    match = regexes[regex_string].search(text)
    return match.group(1).strip() if match else None
 
# Inicializar lista para armazenar os dados
data = []

# Iterar sobre as páginas do dicionário texto_paginas
for page_number, page_text in texto_paginas.items():
    # Extrair informações de cada página usando regex
    row = {
        'pagina_pdf': page_number,
        'nome': get_feature(page_text, 'nome'),
        'nif': get_feature(page_text, 'nif'),
        'mes_mapa': get_feature(page_text, 'mes_mapa'),
        'instituicao': get_feature(page_text, 'instituicao'),
        'divida': get_feature(page_text, 'total_em_divida'),
        'litigio': get_feature(page_text, 'litigio'),
        'parcela': get_feature(page_text, 'abatido_ativo'),
        'garantias': get_feature(page_text, 'garantias'),
        'num_devedores': get_feature(page_text, 'num_devedores'),
        'prod_financeiro': get_feature(page_text, 'prod_financeiro'),
        'entrada_incumpr': get_feature(page_text,'entrada_incumpr'),
        'dat_inicio': get_feature(page_text, 'dat_inicio'),
        'dat_fim': get_feature(page_text, 'dat_fim')
    }
    # Adicionar os dados ao conjunto
    data.append(row)

# Criar o DataFrame
df = pd.DataFrame(data)
df.iloc[:-1]

Pasta já existe: ./maps/encrypted/
Pasta já existe: ./maps/encrypted/processed
Pasta já existe: ./maps/decrypted/
Pasta já existe: ./maps/decrypted/processed
PDF desbloqueado com sucesso! Salvo em: ./maps/decrypted/decrypted_A5E785EC044E5AB7.pdf
PDF original criptografado movido para: ./maps/encrypted/processed

Total de páginas detectadas em 'decrypted_A5E785EC044E5AB7.pdf': 8

PDF 'decrypted_A5E785EC044E5AB7.pdf' movido para a pasta de processados: maps/decrypted/processed/decrypted_A5E785EC044E5AB7.pdf


,pagina_pdf,nome,nif,mes_mapa,instituicao,divida,litigio,parcela,garantias,num_devedores,prod_financeiro,entrada_incumpr,dat_inicio,dat_fim
0,texto_pagina1,INES BOTAS LEAL,208484493,outubro de 2023,"BANCO PRIMUS, SA (0246)","4 684,87",Não,"92,32",None,1,Crédito pessoal,None,2022-11-14,2029-12-05
1,texto_pagina2,INES BOTAS LEAL,208484493,outubro de 2023,"BANCO CREDIBOM, SA (0916)","4 806,51",Não,"85,24",None,1,Crédito pessoal,None,2022-11-10,2029-11-23
2,texto_pagina3,INES BOTAS LEAL,208484493,outubro de 2023,"UNICRE - INSTITUIÇÃO FINANCEIRA DE CRÉDITO, S....","2 889,40",Não,None,None,1,Cartão de crédito - com período de free-float,None,2023-03-31,9999-12-31
3,texto_pagina4,INES BOTAS LEAL,208484493,outubro de 2023,"WIZINK BANK, S.A.U. - SUCURSAL EM PORTUGAL (0272)","2 423,11",Não,"0,00",None,1,Cartão de crédito - com período de free-float,None,2023-03-21,9999-12-31
4,texto_pagina5,INES BOTAS LEAL,208484493,outubro de 2023,"UNIVERSO, IME, S.A. (7500)","669,84",Não,"33,45",None,1,Cartão de crédito - com período de free-float,None,2023-07-05,9999-12-31
5,texto_pagina6,INES BOTAS LEAL,208484493,outubro de 2023,ONEY BANK - SUCURSAL EM PORTUGAL (0881),"1 425,32",Não,"0,00",None,1,Cartão de crédito - com período de free-float,None,2023-07-08,9999-12-31
6,texto_pagina7,INES BOTAS LEAL,208484493,outubro de 2023,Hefesto - Sociedade de Titularização de Crédit...,"7 091,92",Sim,"0,00",None,1,Crédito pessoal,2021-04-01,2018-11-22,9999-12-31
